# 지역별 여가 산업 활성화를 위한 여가 활동 특성 분석

## 데이터 전처리

### import modules

In [868]:
import numpy as np
import pandas as pd
import geopandas as gpd
import scipy.stats as stats
from pandas import Series, DataFrame

In [632]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots as ms

In [633]:
import seaborn as sns
import matplotlib.pyplot as plt

### 여가 문화 관련 정성적 데이터 전처리

사용 데이터
- lsr_time : 하루 평균 여가문화 시간 및 사용 비중
- lsr_stf : 여가 생활 여건 및 만족도 평가

In [620]:
lsr_time = pd.read_csv('./data/leisure_time_info.csv')
lsr_stf = pd.read_csv('./data/leisure_satisfied_info.csv')

# 사용하지 않는 열 제거
lsr_time.drop("RESPOND_ID", axis=1, inplace=True)
lsr_stf.drop("RESPOND_ID", axis=1, inplace=True)

# 조사 기간 표기 방법 통일
def transform_month(x):
    dic_month = {"01":"Jen", "02":"Feb", "03":"Mar", "04":"Apr", "05":"May", "06":"Jun", "07":"Jul", "08":"Aug", "09":"Sep", "10":"Oct", "11":"Nov", "12":"Dec"}
    return dic_month[str(x)[4:6]]

lsr_time["EXAMIN_BEGIN_DE"] = lsr_time["EXAMIN_BEGIN_DE"].apply(lambda x:transform_month(x))
lsr_stf["EXAMIN_YM"] = lsr_stf["EXAMIN_YM"].apply(lambda x:transform_month(x))

# 정성적 답변 정량화
def answer_numerate(x):
    quantity = {"매우 그렇다":5, "그런 편이다":4, "보통이다":3, "그렇지 않은 편이다":2, "전혀 그렇지 않다":1,
                "매우 만족한다":5, "만족하는 편이다":4, "불만족하는 편이다":2, "매우 불만족한다":1}
    return quantity[x]

lsr_stf["TIME_SUFNCY_SATSFC_DGREE_NUM"] = lsr_stf["TIME_SUFNCY_SATSFC_DGREE_VALUE"].apply(lambda x:answer_numerate(x))
lsr_stf["INFRA_ND_FCLTY_SATSFC_DGREE_NUM"] = lsr_stf["INFRA_ND_FCLTY_SATSFC_DGREE_VALUE"].apply(lambda x:answer_numerate(x))
lsr_stf["ECNM_MRGN_SATSFC_DGREE_NUM"] = lsr_stf["ECNM_MRGN_SATSFC_DGREE_VALUE"].apply(lambda x:answer_numerate(x))
lsr_stf["INFO_SUFNCY_SATSFC_DGREE_NUM"] = lsr_stf["INFO_SUFNCY_SATSFC_DGREE_VALUE"].apply(lambda x:answer_numerate(x))
lsr_stf["LSR_LVLH_WHOLE_SATSFC_DGREE_NUM"] = lsr_stf["LSR_LVLH_WHOLE_SATSFC_DGREE_VALUE"].apply(lambda x:answer_numerate(x))

### 레저 사업장 좌표 데이터 전처리

사용 데이터 
- lsr_loc = 레저 사업장 위치 정보
- kor_loc = 대한민국 시,도 행정구역에 대한 공간 데이터

In [418]:
from shapely.geometry import Point, Polygon, MultiPolygon

lsr_loc = pd.read_csv('./data/leisure_location_info.csv')

# 대한민국 행정지역 경계 및 면적 데이터
kor_loc = gpd.read_file('./data/ctp_rvn.shp', encoding="cp949")
kor_area = pd.read_csv('./data/ctp_area.csv', encoding="cp949")[16:].reset_index(drop=True)[["남북한별 ", "2021"]]
kor_area.columns = ["CTP_KOR_NM", "CTP_KOR_AREA"]

# folium에서 사용할 수 있는 좌표계로 변환
kor_loc = kor_loc.set_crs(epsg=5179, inplace=True, allow_override=True)
kor_loc["geometry"] = kor_loc["geometry"].to_crs(epsg=4326)

# 각 데이터 프레임에서 좌표만 추출해서 리스트로 반환
lsr_points = [tuple(x) for x in lsr_loc[['FCLTY_CRDNT_LO', 'FCLTY_CRDNT_LA']].to_numpy()]
kor_boundaries = [x for x in kor_loc["geometry"]]
kor_boundaries_name = [[x, 0] for x in kor_loc["CTP_KOR_NM"]]

# 각 행정구역 내에서 레저 사업장의 개수를 count하는 함수
def inner_or_outer(point_list, i):
    for point in point_list.copy():
        if kor_boundaries[i].contains(Point(point)):
            kor_boundaries_name[i][1] += 1
            point_list.remove(point)

for i in range(len(kor_boundaries)):
    inner_or_outer(lsr_points, i)

# count한 결과 데이터프레임으로 반환
lsr_count = pd.DataFrame(kor_boundaries_name)
lsr_count.columns = ["CTP_KOR_NM", "NUM_OF_LSR_LOC"]

# 면적당 레저 사업장 개수 반환
lsr_locations = pd.merge(lsr_count, kor_area, on="CTP_KOR_NM")
lsr_locations["NUM_OF_LSR_LOC"] = lsr_locations["NUM_OF_LSR_LOC"].astype('int')
lsr_locations["CTP_KOR_AREA"] = lsr_locations["CTP_KOR_AREA"].astype('int')
lsr_locations["LSR_LOC_PER_AREA"] = lsr_locations["NUM_OF_LSR_LOC"].div(lsr_locations["CTP_KOR_AREA"])*100

- 함수 실행시에 좌표 104개가 빠지는데, 시각화 결과 38개는 좌표값이 아프리카 바다에.. 덩그러니 있고 나머지 66개는.. 어디에 갔는지 모르겠서욤 ㅜ

In [634]:
lsr_time

,EXAMIN_BEGIN_DE,SEXDSTN_FLAG_CD,AGRDE_FLAG_NM,ANSWRR_OC_AREA_NM,HSHLD_INCOME_DGREE_NM,WORKDAY_DAY_AVRG_LSR_TIME_VALUE,WKEND_DAY_AVRG_LSR_TIME_VALUE,ONE_WEEK_TOT_LSR_TIME_VALUE,LSR_TIME_REST_RCRT_USE_RATE,LSR_TIME_HOBBY_USE_RATE,LSR_TIME_SELF_IMPT_USE_RATE,LSR_TIME_TWDPSN_RLTN_FLWSP_USE_RATE,LSR_TIME_ETC_USE_RATE
0,Jen,F,50대,광주광역시,300이상500만원 미만,2,3,16,40,0,0,60,0
1,Jen,M,60대,전라남도,300만원 미만,3,5,25,0,0,0,30,70
2,Jen,F,30대,경기도,300이상500만원 미만,3,12,39,60,40,0,0,0
3,Jen,M,50대,대전광역시,300만원 미만,3,3,21,0,0,25,65,10
4,Jen,F,30대,경기도,500이상700만원 미만,3,12,39,50,0,20,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26973,Dec,F,40대,경상북도,500이상700만원 미만,2,1,12,0,100,0,0,0
26974,Dec,M,20대,광주광역시,700만원 이상,7,7,49,60,10,20,10,0
26975,Dec,M,50대,광주광역시,300만원 미만,10,10,70,90,0,0,0,10
26976,Dec,F,20대,경기도,300이상500만원 미만,5,7,39,40,5,10,0,45


In [420]:
lsr_stf

,EXAMIN_YM,SEXDSTN_FLAG_CD,AGRDE_FLAG_NM,ANSWRR_OC_AREA_NM,HSHLD_INCOME_DGREE_NM,TIME_SUFNCY_SATSFC_DGREE_VALUE,INFRA_ND_FCLTY_SATSFC_DGREE_VALUE,ECNM_MRGN_SATSFC_DGREE_VALUE,INFO_SUFNCY_SATSFC_DGREE_VALUE,LSR_LVLH_WHOLE_SATSFC_DGREE_VALUE,TIME_SUFNCY_SATSFC_DGREE_NUM,INFRA_ND_FCLTY_SATSFC_DGREE_NUM,ECNM_MRGN_SATSFC_DGREE_NUM,INFO_SUFNCY_SATSFC_DGREE_NUM,LSR_LVLH_WHOLE_SATSFC_DGREE_NUM
0,Jen,M,20대,강원도,300만원 미만,보통이다,그런 편이다,보통이다,그런 편이다,만족하는 편이다,3,4,3,4,4
1,Jen,F,30대,강원도,300만원 미만,그렇지 않은 편이다,그렇지 않은 편이다,보통이다,그런 편이다,보통이다,2,2,3,4,3
2,Jen,F,30대,강원도,300만원 미만,그런 편이다,매우 그렇다,그렇지 않은 편이다,그렇지 않은 편이다,보통이다,4,5,2,2,3
3,Jen,M,40대,강원도,300만원 미만,보통이다,그런 편이다,보통이다,보통이다,보통이다,3,4,3,3,3
4,Jen,M,30대,강원도,300만원 미만,그런 편이다,그렇지 않은 편이다,보통이다,그런 편이다,매우 만족한다,4,2,3,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26973,Dec,F,40대,경상북도,500이상700만원 미만,그런 편이다,보통이다,보통이다,보통이다,보통이다,4,3,3,3,3
26974,Dec,M,20대,광주광역시,700만원 이상,보통이다,그런 편이다,보통이다,그런 편이다,만족하는 편이다,3,4,3,4,4
26975,Dec,M,50대,광주광역시,300만원 미만,보통이다,보통이다,보통이다,보통이다,보통이다,3,3,3,3,3
26976,Dec,F,20대,경기도,300이상500만원 미만,그런 편이다,그런 편이다,그런 편이다,그런 편이다,만족하는 편이다,4,4,4,4,4


In [ ]:
lsr_locations

,CTP_KOR_NM,NUM_OF_LSR_LOC,CTP_KOR_AREA,LSR_LOC_PER_AREA
0,서울특별시,3159,605,522.148760
1,부산광역시,654,770,84.935065
2,대구광역시,770,884,87.104072
3,인천광역시,344,1066,32.270169
4,광주광역시,74,501,14.770459
5,대전광역시,132,540,24.444444
6,울산광역시,56,1062,5.273070
7,세종특별자치시,24,465,5.161290
8,경기도,3774,10197,37.010886
9,강원도,1717,16830,10.202020


## 가설1. 거주 지역이 사람들의 여가 생활에 영향을 줄 것이다.

In [521]:
df_time_1 = lsr_time.copy()
df_loc_1 = lsr_locations.copy()
df_stf_1 = lsr_stf.copy()
df = pd.merge(df_loc_1, df_time_1, right_on="ANSWRR_OC_AREA_NM", left_on="CTP_KOR_NM")

In [924]:
map = folium.Map(location=[37.509671, 127.055517], zoom_start=8)
loc = lsr_loc[["FCLTY_CRDNT_LA", "FCLTY_CRDNT_LO"]]

marker_cluster = MarkerCluster().add_to(map)
for la, lo in zip(loc["FCLTY_CRDNT_LA"], loc["FCLTY_CRDNT_LO"]):
    folium.Marker([la, lo], icon=folium.Icon(color="green")).add_to(marker_cluster)
folium.GeoJson(kor_loc).add_to(map)

map

NameError: name 'ƒsr' is not defined

#### 가설 1-1. 레저 사업장이 다수 위치한 곳일수록 주민들의 인프라 및 시설 만족 정도가 높을 것이다

레저 사업장의 위치 분포

In [780]:
fig = px.bar(df_loc_1.sort_values(by="NUM_OF_LSR_LOC", ascending=False), x="CTP_KOR_NM", y="NUM_OF_LSR_LOC", color="CTP_KOR_NM", text='NUM_OF_LSR_LOC')
fig.update_layout(title = "지역별 레저 사업장 개수")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

In [781]:
fig = px.bar(df_loc_1.sort_values(by="LSR_LOC_PER_AREA", ascending=False), x="CTP_KOR_NM", y="LSR_LOC_PER_AREA", color="CTP_KOR_NM", text='LSR_LOC_PER_AREA')
fig.update_layout(title = "지역별 100km² 면적당 레저 사업장 개수")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

100km²당 레저 사업장 개수를 토대로 서울, 제주, 대구, 부산 등에 레저 사업장이 다수 위치해 있음을 확인할 수 있다. </br>
이를 보며 레저 사업장이 많이 위치한 지역일수록 도로, 통신 등의 인프라가 좋을 것이라고 판단하였고, 따라서 위와 같은 가설을 설정하였다.

In [869]:
stf_infra_mean = df_stf_1.groupby("ANSWRR_OC_AREA_NM")["INFRA_ND_FCLTY_SATSFC_DGREE_NUM"].agg(np.mean).reset_index()
stf_infra_mean.rename(columns = {'INFRA_ND_FCLTY_SATSFC_DGREE_NUM' : 'INFRA_SATSFC_DGREE_MEAN'}, inplace = True)

df1 = pd.merge(df_loc_1, stf_infra_mean, left_on="CTP_KOR_NM", right_on="ANSWRR_OC_AREA_NM")[["ANSWRR_OC_AREA_NM", "INFRA_SATSFC_DGREE_MEAN", "LSR_LOC_PER_AREA"]].sort_values(by="LSR_LOC_PER_AREA", ascending=False)

In [875]:
px.imshow(df1[["LSR_LOC_PER_AREA", "INFRA_SATSFC_DGREE_MEAN"]].corr(),
                color_continuous_scale = px.colors.diverging.RdBu, text_auto=True,
                color_continuous_midpoint = 0, height = 500,
                title = "레저 사업장의 개수와 인프라 및 시설 만족 정도에 대한 상관관계분석")

In [876]:
stats.pearsonr(df1.LSR_LOC_PER_AREA.values, df1.INFRA_SATSFC_DGREE_MEAN.values)[1] # p-value로 상관 계수의 유효성 파악

0.011082139990820037

시∙도민의 인프라 및 시설 만족 정도 설문 조사 결과의 평균 값을 기준으로 하여 레저 사업장의 개수와의 상관 관계를 분석하였을 때, 상관 계수가 0.59886으로 양의 값이 나온다. </br>
따라서 레저 사업장의 개수가 많은 지역일수록 시∙도민의 인프라 및 시설에 대한 만족 정도가 높음을 확인할 수 있다. </br>
해당 상관관계분석에서 p-value는 0.01108으로 0.05보다 작으므로 본 상관관계분석결과는 의미가 있다고 볼 수 있다.

#### 가설 1-2. 레저 사업장이 다수 위치한 곳에 거주하는 사람일수록 그렇지 않은 사람보다 비수기에 여가 활동을 더 많이 할 것이다.

가설 1-1에서 레저 사업장의 개수가 많은 지역일수록 시∙도민의 인프라 및 시설에 대한 만족 정도가 높음을 파악했다. </br>
이를 기반으로, 레저 사업장이 다수 위치한 사람은 레저 사업장에 대한 접근성이 좋을 것이라고 추측할 수 있다. </br>
따라서 시간적 여유로움이 덜한 비수기에도 레저 활동을 더 많이 할 수 있을 것이라고 예상한다.

- 여가 활동의 성수기는 일반적인 휴가철인 7~8월, 12월~1월로 정의한다.

In [700]:
# 비수기로 기간 한정
df3 = df_time_1[(df_time_1["EXAMIN_BEGIN_DE"]!="Jul") & (df_time_1["EXAMIN_BEGIN_DE"]!="Aug") & (df_time_1["EXAMIN_BEGIN_DE"]!="Dec") & (df_time_1["EXAMIN_BEGIN_DE"]!="Jen")]

# 이상치 제거
level_1q = df3["ONE_WEEK_TOT_LSR_TIME_VALUE"].quantile(0.25)
level_3q = df3["ONE_WEEK_TOT_LSR_TIME_VALUE"].quantile(0.75)
IQR = level_3q - level_1q
rev_range = 1.5

# 통계치 계산
off_season = df3[(df3["ONE_WEEK_TOT_LSR_TIME_VALUE"] <= level_3q + (rev_range * IQR)) & (df3['ONE_WEEK_TOT_LSR_TIME_VALUE'] >= level_1q - (rev_range * IQR))][["ANSWRR_OC_AREA_NM", "EXAMIN_BEGIN_DE", "ONE_WEEK_TOT_LSR_TIME_VALUE"]]
off_statistic = off_season.groupby(["ANSWRR_OC_AREA_NM"])["ONE_WEEK_TOT_LSR_TIME_VALUE"].agg([np.mean]).reset_index()

# 레저 사업장 개수 df와 병합
df4 = pd.merge(df_loc_1, off_statistic, left_on="CTP_KOR_NM", right_on="ANSWRR_OC_AREA_NM")
df4.rename(columns = {'mean' : 'OFF_LSRT_TIME_MEAN'}, inplace = True)

In [707]:
px.scatter(df4, x="LSR_LOC_PER_AREA", y="OFF_LSRT_TIME_MEAN", color="ANSWRR_OC_AREA_NM", size='NUM_OF_LSR_LOC')

In [873]:
px.imshow(df4[["LSR_LOC_PER_AREA", "OFF_LSRT_TIME_MEAN"]].corr(),
                color_continuous_scale = px.colors.diverging.RdBu, text_auto=True,
                color_continuous_midpoint = 0, height = 500,
                title = "레저 사업장의 개수와 비수기의 여가 활동 시간에 대한 상관관계분석")

In [874]:
stats.pearsonr(df4.LSR_LOC_PER_AREA.values, df4.OFF_LSRT_TIME_MEAN.values)[1] # p-value로 상관 계수의 유효성 파악

0.030644571304248178

위의 그래프를 보면 일반적으로 x축의 값(레저 사업장의 개수)이 증가할수록 y축의 값(비수기 여가 활동 시간)도 증가하는 경향을 보인다. </br>
상관계수를 구했을 때 0.52453로 0보다 큰 값을 가지므로 양의 상관관계를 가지며, p-value가 0.03064로 0.05보다 작은 값이기 때문에, 해당 상관계수는 의미를 가진다. </br>


따라서 분석결과 레저 사업장의 개수가 많을수록 비수기의 여가 활동 시간도 늘어난다고 할 수 있다.

그러나, 제주특별자치도민의 경우 해당 지역에 서울특별시 다음으로 면적당 지역 사업장의 개수가 많음에도 불구하고 비수기에 여가 활동을 거의 하지 않는 것으로 보여진다. </br>
이에 대한 원인이 무엇인지에 대해서는 별도의 추가적인 분석이 필요하다.

#### 제주도 원인 분석

In [894]:
df_jeju = df_stf_1[df_stf_1["ANSWRR_OC_AREA_NM"]=="제주특별자치도"]
df_seoul = df_stf_1[df_stf_1["ANSWRR_OC_AREA_NM"]=="서울특별시"][["ANSWRR_OC_AREA_NM", "INFO_SUFNCY_SATSFC_DGREE_NUM", "INFO_SUFNCY_SATSFC_DGREE_VALUE"]]

In [923]:
df5 = df_jeju[["TIME_SUFNCY_SATSFC_DGREE_NUM", "INFRA_ND_FCLTY_SATSFC_DGREE_NUM", "ECNM_MRGN_SATSFC_DGREE_NUM", "INFO_SUFNCY_SATSFC_DGREE_NUM", "LSR_LVLH_WHOLE_SATSFC_DGREE_NUM"]].agg(np.mean).reset_index()
df5.columns = ["SURVERY_QUE", "ANS_MEAN"]

px.bar(df5.sort_values(by="ANS_MEAN"), x="SURVERY_QUE", y="ANS_MEAN", color="SURVERY_QUE", text='ANS_MEAN')

제주도민의 여가생활 여건 및 만족도 평가 내용을 보면 평균적으로 경제 여유 만족 정도 값과 정보 충분 만족 정도값이 낮은 것을 확인할 수 있다.

In [779]:
# 서울특별시와 제주특별자치도 정보 충분 만족도 응답 비교
fig = ms(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df_jeju.INFO_SUFNCY_SATSFC_DGREE_VALUE.values, values=df_jeju.INFO_SUFNCY_SATSFC_DGREE_NUM.values, title="제주특별자치도"), 1, 1)
fig.add_trace(go.Pie(labels=df_seoul.INFO_SUFNCY_SATSFC_DGREE_VALUE.values, values=df_seoul.INFO_SUFNCY_SATSFC_DGREE_NUM.values, title="서울특별시"), 1, 2)
fig.update_traces(hole=.4, hoverinfo="label+percent+name")
fig.update_layout(title_text="제주특별자치도와 서울특별시의 정보 충분 만족도 응답 비교")
fig.show()

In [916]:
# 시∙도민의 정보충분만족정도에 대한 평균
df6 = df_stf_1.groupby("ANSWRR_OC_AREA_NM")["INFO_SUFNCY_SATSFC_DGREE_NUM"].agg(np.mean).reset_index()
df6.rename(columns = {'INFO_SUFNCY_SATSFC_DGREE_NUM' : 'INFO_SUFNCY_SATSFC_DGREE_MEAN'}, inplace = True)

fig = px.bar(df6.sort_values(by="INFO_SUFNCY_SATSFC_DGREE_MEAN"), x="ANSWRR_OC_AREA_NM", y="INFO_SUFNCY_SATSFC_DGREE_MEAN", color="ANSWRR_OC_AREA_NM", text='INFO_SUFNCY_SATSFC_DGREE_MEAN')
fig.update_layout(title = "지역별 정보 충분 만족도에 대한 평균")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()